# Validate Land Surface Temperature (LST) using interactive widgets

## Import libraries and define functions

### Import libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os

import numpy as np
import pandas as pd
import geopandas as gpd
import json

import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as ipyw
import ipyleaflet as ipyl
import branca.colormap as cm

from scipy.stats import pearsonr
from itertools import combinations
from copy import deepcopy
from datetime import datetime, date

# Local imports
import land_surface_temperature_retrieval.utils_dashboard as process

### Define Functions:
How should the subplots be plotted:

In [3]:
def create_choropleth(df, key_column, value_column, metric):
    """ Create an ipyleaflet choropleth layer, from a gpd.GeoDataFrame by defining the key and value column """
    
    # Generate a colormap for the choropleth, based on defined metric
    if metric == "mae":
        vmin, vmax = 1.0, 5.0 #user defined vmin/vmax
        cmap = cm.LinearColormap(colors=["#fff5f0", "#fdbea5", "#fc7050", "#d42020", "#67000d"],
                                 index=process.interpolate_linear(vmin=vmin, vmax=vmax), vmin=vmin, vmax=vmax,
                                 caption='mae colormap').to_step(5)
    if metric == "bias":
        vmin, vmax = -5.0, 5.0 #user defined vmin/vmax
        cmap = cm.LinearColormap(colors=["#0571b0", "#92c5de", "#f7f7f7", "#f4a582", "#ca0020"],
                                 index=process.interpolate_linear(vmin=vmin, vmax=vmax, center=0), vmin=vmin, vmax=vmax,
                                 caption='bias colormap').to_step(5)
    if metric == "ub_rmse":
        vmin, vmax = 0.0, 5.0 #user defined vmin/vmax
        cmap = cm.LinearColormap(colors=["#fff5f0", "#fdbea5", "#fc7050", "#d42020", "#67000d"],
                                 index=process.interpolate_linear(vmin=vmin, vmax=vmax), vmin=vmin, vmax=vmax,
                                 caption='ubRMSE colormap').to_step(5)
    if metric == "r":
        vmin, vmax = 0.7, 1.0
        cmap = cm.LinearColormap(colors=["#f7fcf5", "#caeac3", "#7bc87c", "#2a924a", "#00441b"],
                                 index=process.interpolate_log(vmin=vmin, vmax=vmax), vmin=vmin, vmax=vmax,
                                 caption='Pearson r colormap').to_step(5)

    df = df.dropna() # NaNs are not understood in the chorpleth, so make sure there are none
    
    geo_json = json.loads(df[["geometry", key_column, value_column]].to_json()) # convert to a GeoJSON
    choro_data = dict(zip(list(map(str, df.index.values)), df[value_column].tolist())) # Create dict with values, for which classes will be created.
    geo_json_map = ipyl.Choropleth(geo_data=geo_json, # Plot the choropleth markers
                                   choro_data=choro_data, 
                                   colormap=cmap, 
                                   key_on="id",
                                   value_min=vmin,
                                   value_max=vmax,
                                   hover_style={'color': 'black', 'fillColor': "grey", 'opacity':1, 'fillOpacity': 1},
                                   point_style={'radius': 8, 'fillOpacity': 1, 'weight': 3})
    
    return geo_json_map, vmin, vmax


def PlotTimeSeries(stations_df, station, selected_products, t0=datetime(2018, 1, 1), t1=datetime(2018, 12, 31)):
    """ Create a matplotlib figure of the selected time products"""
    # cmap to ensure scatter and line of same product always have the same color
    cm = plt.get_cmap('tab20')
    
    # From the selected products, only plot the ones where the moving average has been taken from:
    products = [i for i in selected_products] + ["roll_"+i for i in selected_products]
    
    # Initiate Figure
    fig, ax = plt.subplots(figsize=(12,7))
    
    # Prepare data to be plotted:
    row = stations_df.loc[stations_df["name"] == station]
    data = dict(zip(products, row[products].iloc[0]))
    df = pd.DataFrame(index=row["date_time_local"].iloc[0], data=data, columns=products)
    
    # Plot the actual values as scatter:
    i=0
    for product, label in zip(products, selected_products):
        df.plot(ax=ax, use_index=True, y=product, label='_nolegend_', style='.', color=cm(i+1), legend=False)
        i=i+2
    
    # Plot the rolled averages as line:
    i=0
    for product, label in zip(products, selected_products):
        df.plot(ax=ax, use_index=True, y="roll_"+product, label=label, color=cm(i), legend=True)
        i=i+2    
    
    # Modify some visualisation settings:
    ax.set_xlim(min(row["date_time_local"].iloc[0]), max(row["date_time_local"].iloc[0]))
    plt.axvspan(t0, t1, color='grey', alpha=0.2)
    ax.set_ylabel("Temperature ($^\circ$C)")
    ax.set_title(station)
    ax.grid()
    ax.legend(loc='upper left')
    fig.tight_layout()
    
    return fig


def PlotHeatmap(df, station="", sources = [], mode="dynamic", **kwargs):
    # Determine Evaluation Metrics for defined station
    if mode == "dynamic":
        table = process.StationTable(df=df, station_name=station, products=sources) #df.loc[station].transpose()
    elif mode == "static":
        table = df.groupby(level=1).mean().transpose() # Spatially averaged metrics
    
    # Initiate Figure
    fig, (ax0, ax1, ax2, ax3) = plt.subplots(nrows=4, ncols=1, figsize=(12,4), sharex=True) 
    
    # Define tick labels
    yt = ["Pearson r", "Average\nbias", "MAE", "ubRMSE"]
    cols = table.columns.to_list()
    cols = [col.replace(" x ", "\nx\n") for col in cols]
    if mode == "dynamic": # Add n observations as well (for dynamic table)
        values = list(table.iloc[-1,:])
        cols = [col + f"\n(n={str(int(value))})" for col, value in zip(cols, values)]
    else:
        cols = [col + "\n" for col in cols] # This step is for table allignment purposes
        
    
    # Plot on each ax, and define some visualisation settings
    sns.heatmap(data = pd.DataFrame(table.iloc[0,:]).transpose(), ax = ax0, 
                annot = True, cbar = False, fmt = ".2f", linewidths = 0.5, yticklabels=[yt[0]], xticklabels=cols,
                cmap="Reds_r", vmin=0.7, vmax=1,annot_kws={"size": 16, "weight": "bold"})
    sns.heatmap(data = pd.DataFrame(table.iloc[1,:]).transpose(), ax = ax1, 
                annot = True, cbar = False, fmt = ".2f", linewidths = 0.5, yticklabels=[yt[1]], xticklabels=cols,
                cmap="RdBu_r", center=0,annot_kws={"size": 16, "weight": "bold"})
    sns.heatmap(data = pd.DataFrame(table.iloc[2,:]).transpose(), ax = ax2, 
                annot = True, cbar = False, fmt = ".2f", linewidths = 0.5, yticklabels=[yt[2]], xticklabels=cols,
                cmap="Reds", vmin=0,annot_kws={"size": 16, "weight": "bold"})
    sns.heatmap(data = pd.DataFrame(table.iloc[3,:]).transpose(), ax = ax3, 
                annot = True, cbar = False, fmt = ".2f", linewidths = 0.5, yticklabels=[yt[3]], xticklabels=cols,
                cmap="Reds", vmin=0,annot_kws={"size": 16, "weight": "bold"})
    
    for ax in [ax0, ax1, ax2, ax3]: # Rotate the labels on each axis, so they appear 'nicer'
        ax.set_yticklabels(ax.get_yticklabels(),rotation = 0, fontsize = 16)
        ax.set_xticklabels(ax.get_xticklabels(), fontsize = 16)

    return fig



def create_legend(vmin, vmax, metric):
    """Create a colormap legend for (return: ipyleaflet.LegendControl) """

    # Determine corresponding colors based on chosen metric
    if metric == "bias":
        values = process.interpolate_linear(vmin, vmax)
        cmap = ["#0571b0", "#92c5de", "#f7f7f7", "#f4a582", "#ca0020"] # Manual list of bwr colormap!
    elif metric == "mae":
        values = process.interpolate_linear(vmin, vmax)
        cmap = ["#fff5f0", "#fdbea5", "#fc7050", "#d42020", "#67000d"] # Manual list of reds colormap!
    elif metric == "ub_rmse":
        values = process.interpolate_linear(vmin, vmax)
        cmap = ["#fff5f0", "#fdbea5", "#fc7050", "#d42020", "#67000d"] # Manual list of reds colormap!
    elif metric == "r":
        values = process.interpolate_log(0.7, 1.0)
        cmap = ["#f7fcf5", "#caeac3", "#7bc87c", "#2a924a", "#00441b"] # Manual list of greens colormap!
        
    values = [str(value) for value in values]
    
    return ipyl.LegendControl(dict(zip(values, cmap)), 
                              name=f"{metric}", 
                              position="bottomleft")

Define the widgets displayed in the dashboard:

In [4]:
def create_plot_as_widget(station, products, start_date, end_date):
    """create timeseries as widget"""
    return ipyw.interactive_output(PlotTimeSeries,
                                   {'stations_df': ipyw.fixed(data),
                                    'station': ipyw.fixed(station),
                                    'selected_products' : ipyw.fixed(products),
                                    't0' : ipyw.fixed(start_date),
                                    't1' : ipyw.fixed(end_date)})


def create_dynamic_table_as_widget(station, df):
    """create dynamic table as widget"""
    return ipyw.interactive_output(PlotHeatmap,
                                   {'df': ipyw.fixed(df),
                                    'station': ipyw.fixed(station),
                                    'sources': ipyw.fixed(products),
                                    'mode': ipyw.fixed("dynamic")})


def create_static_table_as_widget():
    """create static table as widget"""
    return ipyw.interactive_output(PlotHeatmap,
                                   {'df': ipyw.fixed(evaluation_df),
                                    'sources': ipyw.fixed(products),
                                    'mode': ipyw.fixed("static")})

Define the callback after user interactions:

In [5]:
# ===== Changes to choropleth =====
def update_choropleth(df, new_left, new_right, new_metric):
    """ Update the choropleth and legend layers, depending on the change defined in update_choropleth_product() """
    
    global left_product, right_product, metric # Any update that happens to the data df, left, right or metric variables, should also exist outside this function
    
    old_choropleth = m.layers[-1] # Identify the old Choropleth layer
    

    if new_left == new_right: # Avoid Comparing the same thing
        print("left and right products cannot be the same! \nNo change applied!")
        return
    
    try: # First check if left/right combination exists
        value = new_metric+"_"+f"{new_left} x {new_right}"
        new_choropleth, vmin, vmax = create_choropleth(df=data, key_column="name", value_column=value, metric=new_metric) # Create new choropleth
    except: # If not, check if right/left combination exists
        value = new_metric+"_"+f"{new_right} x {new_left}"
        inverted_df = data.copy() # To make sure choropleth is correct, invert values of selected column
        if new_metric == "bias": 
            inverted_df[value] = inverted_df[value] * -1.
        if new_metric == "ub_rmse":
            inverted_df = process.inverted_ubrmse(data=inverted_df, left=new_left, right=new_right) # Recalculate ubRMSE
        new_choropleth, vmin, vmax = create_choropleth(df=inverted_df, key_column="name", value_column=value, metric=new_metric) # Create new choropleth

    legend = create_legend(vmin, vmax, metric=new_metric) # create new legend

    new_choropleth.on_click(on_click) # Make sure new choropleth is interactive  
    
    m.substitute_layer(old_choropleth, new_choropleth) # Replace old with new choropleth
    m.remove_control(m.controls[-1]) # Remove old legend
    m.add_control(legend) # Add new legend
    
    print(f"left: {new_left}, right: {new_right}")
    left_product, right_product, metric = new_left, new_right, new_metric # Make sure the global variables are updated

    return


def update_choropleth_product(change):
    """ Update the choropleth and legend layers, when left product, right product or evaluation metric is changed """
    
    if change["owner"].description == 'Metric:':
        update_choropleth(df=data, new_left=left_product, new_right=right_product, new_metric=change["new"])
    elif change["owner"].description == "Left Product:":
        d = dict(change["owner"].options) # Identify the dict with id's (integers) and labels of dropdown menu
        label = (list(d.keys())[list(d.values()).index(change["new"])]) # Get the label corresponds to the selected id
        if label == right_product:
            print("The left and right products cannot be the same!")
            return
        update_choropleth(df=data, new_left=label, new_right=right_product, new_metric=metric) # Update the chloropeth
    elif change["owner"].description == "Right Product:":
        d = dict(change["owner"].options) # Identify the dict with id's (integers) and labels of dropdown menu
        label = (list(d.keys())[list(d.values()).index(change["new"])]) # Get the label corresponds to the selected id
        if label == left_product:
            print("The left and right products cannot be the same!")
            return
        update_choropleth(df=data, new_left=left_product, new_right=label, new_metric=metric) # Update the chloropeth 
    return


# ===== Changes to timeseries =====
def on_click(feature=None, **kwargs):
    """ Update timeseries, when a station is clicked """
    
    global station # Make sure that current station is updated outside this function as well, so future changes can rely on the most up to date values.
    
    try:
        station = feature["properties"]["name"] # On click, identify the station
        print(feature["properties"]["name"]) # Log which station has been clicked on
        print(time_filtered_df[time_filtered_df["name"] == station][["elevation", "climate"]]) # Log station characteristics
        plot = create_plot_as_widget(station, products, start_date, end_date) # On click, create timeseries plot
        tab_dynamic = create_dynamic_table_as_widget(station=station, df=time_filtered_df) # On click, create dynamic table
        rbox.children = (rbox.children[0], rbox.children[1], plot, tab_dynamic)
    except TypeError:  # feature is None
        pass


def update_timeseries_products(change):
    """ Update the timeseries plot, when product selection changes """
    
    global products # Make sure that 'products' are updated outside this function as well, so future changes can rely on the most up to date values.
    
    # First check if a feature should be added or removed, then add or remove it
    if change != change:
        pass
    elif change["new"]:
        products.append(change["owner"].description)
    elif not change["new"] and len(products) > 1: # Make sure that the products list never gets smaller than 1, else there is nothing to plot!
        products.remove(change["owner"].description)
    else:
        print("At least one item must be selected!")
        return
    
    products = list(pd.Series(products).unique()) # Make sure there are no duplicate items in list
    
    try:
        plot = create_plot_as_widget(station, products, start_date, end_date) # recreate timeseries plot
        if len(products) > 1:
            time_filtered_df = process.TimeFilter(df=data, start_date=np.datetime64(start_date), end_date=np.datetime64(end_date)) # Update time filtered df
            tab_dynamic = create_dynamic_table_as_widget(station, df=time_filtered_df) # Update dynamic table
            rbox.children = (rbox.children[0], rbox.children[1], plot, tab_dynamic)
        else: rbox.children = (rbox.children[0], rbox.children[1], plot, rbox.children[-1])
    except:
        print("Something went wrong creating the plot!")
        pass

    return


def update_timeseries_dates(feature=None):
    """ Update the timeseries plot, when date selection changes """
    
    global start_date, end_date, time_filtered_df # Any update that happens to the start_date, end_date or time_filtered_df, should also exist outside this function
    
    if feature["owner"].description == "Start Date:":
        start_date = feature["new"]
        print(feature)
    elif feature["owner"].description == "End Date:":
        end_date = feature["new"]
        print(feature)

    plot = create_plot_as_widget(station, products, start_date=start_date, end_date=end_date) # Recreate timeseries plot
    time_filtered_df = process.TimeFilter(df=data, start_date=np.datetime64(start_date), end_date=np.datetime64(end_date)) # Update time filtered df
    tab_dynamic = create_dynamic_table_as_widget(station=station, df=time_filtered_df) # Create dynamic table
    rbox.children = (rbox.children[0], rbox.children[1], plot, tab_dynamic)
    rbox
    
    return


def switch_dataframe(change):
    """ When modis is clipped, """
    global data, data_backup
    
    if change["new"]: # If selected
        data_backup = gpd.GeoDataFrame(columns=data.columns, data=deepcopy(data.values))
        data = process.clip_to_col(data, products=products, clipping_col="T_modis")
    elif not change["new"]:
        data = data_backup
    
    # Update timeseries and dynamic table, but as no selection changes are made, set the change dict to 'nan'
    update_timeseries_products(change=np.nan)

    return


## Import and reformat the input data (.csv)
Define where the input .csv is stored, and also define column names such as the ones with temperature in Kelvin, and those with temperature in Celsius, the date/time column and ones with station specific characteristics (e.g. geometry, station name or elevation). 

Then, the input .csv is read and the data is pre-processed to the format required for the dashboard. For example, date time columns are converted to datetime.datetime format and filtered to a specified hour, temperature is filtered between -40 and +50 degrees celsius and, if desired, a rolling mean function is applied to the data. The final output is GeoDataFrame. 

In [6]:
# ===== Import Data 1 - Change These Parameters =====
# Define data path
data_path = os.path.abspath(os.path.join(os.path.dirname( os.getcwd() ), '.', 'data'))
ismn_filename = 'data_jupyter_notebook.csv'

# Define the column names in the input csv (keys) and their corresponding label to be used (values)
dict_col_rename = {"surface_temperature":            "T_uscrn",
                   "LST_MODIS_V6.1_1000":            "T_modis",
                   #"TEFF-AMSR2-DESC_V003_100":       "T_PlanetV3",
                   "TEFF-AMSR2-DESC_V4.0_1000":      "T_PlanetV4",
                   "LST-S3B-SLSTR-L2-ASC_V1.0_1000": "T_S3" # For shifted values
                  }

# Define some more column characteristics:
celsius_cols = []#["T_uscrn"] # cols with T in celsius
kelvin_cols = ["T_modis", "T_PlanetV4", "T_S3", "T_uscrn"] # cols with T in Kelvin ("T_PlanetV3",)
date_time_col = "date_time_local"
station_info_cols = ["geometry", "name", "elevation", "climate", "landcover"] # cols with station specific characteristics (should at leas contain 'geometry' and 'name')


# ===== Import Data 2 - Do NOT Change These Parameters =====
data = process.open_csv(in_csv=os.path.join(data_path, ismn_filename), date_time_col = "date_time_local")
data.rename(columns=dict_col_rename, inplace=True)
data = process.reformat_per_station(data=data,
                                        kelvin_cols=kelvin_cols,
                                        celsius_cols=celsius_cols,
                                        station_info_cols=station_info_cols,
                                        date_time_col=date_time_col,
                                        hour=2,
                                        T_min=-40,
                                        T_max=50,
                                        add_rolling = True,
                                        window=20,
                                        min_periods=3
                                        )
data = process.TemperatureFilter(df=data, products=celsius_cols+kelvin_cols, lower_lim=0, upper_lim=1e10)

## Define Initial Variables and Pre-process some variables
Set the initial states of the dashboard by defining some variables in the cell below. In the second section of the initial variables, the evaluation metrics are pre-calculated for the choropleth map, and the timeseries are clipped to the desired start- and end-date.

In [7]:
# ===== Initial variables 1 - Change These =====
products = ["T_uscrn", "T_modis", "T_PlanetV4", "T_S3"] # All products we want to consider ("T_PlanetV3",)
station = "Aberdeen-35-WNW" # Station for initial plot
left_product = products[2] # Left product to compare in choropleth map
right_product = products[0] # Right product to compare in choropleth map
metric = "mae" # Metric to display in map in initial rendering. Choose from: "r", "bias", "ub_rmse" or "mae"
start_date = datetime(2020, 9, 1) # Start date of window to calculate metrics over
end_date = datetime(2021, 9, 1) # End date of window to calculate metrics over


# ===== Initial variables 2 - Do NOT Change These =====
evaluation_df = process.EvaluationMetrics(df=data, products=products) # multi-index df with all metrics for all stations
data = process.AppendAllMetricCombinations(data, evaluation_df, products, metrics=["r","bias","mae","ub_rmse","n"]) # Append all possible product combinations and metric combinations to the 'data' df
dynamic_table = evaluation_df.loc[station].transpose() # Metrics of single station

time_filtered_df = process.TimeFilter(data, start_date, end_date) # data df, but arrays filtered to the selected time window

## Plot the interactive dashboard

Define the dashboard layout (e.g. widgets and buttons), the dasboard behavior (desired callbacks for each button/widget) and plot the dashboard. 

Note that the timeseries are depicted as the moving average, while the Pearson r, bias and ubRMSE metrics have been determined element-wise. The metrics in the dynamic table (bottom right) are calculated over the selected time window (grey area in timeseries). Since products might have a different amount of valid observations (n), some metrics might not be representative. As a result, the 'clip to modis' option can be utilized as modis is the product that usually has fewest valid observations.

In [8]:
plt.rcParams['font.size'] = 16 # Define the font size of all plots

# ===== Initialize map =====
m = ipyl.Map(basemap=ipyl.basemaps.OpenTopoMap,
             center=(38, -96),
             zoom=4,
             layout={'width': '750px', 'height': '450px'},
             scroll_wheel_zoom=False,
             zoom_control=False)
m.add_control(ipyl.ZoomControl(position='topright'))
m.add_control(ipyl.ScaleControl(position='bottomleft'))
m.add_control(ipyl.FullScreenControl())


# ===== Initialize buttons, dropdown menu's and date pickers =====
metric_buttons = ipyw.ToggleButtons(options=['r','bias', 'mae', 'ub_rmse'], value=metric, description='Metric:', disabled=False, button_style='')
dropdown_left = ipyw.Dropdown(options=list(zip(products, range(1, len(products)+1))), value=products.index(left_product)+1, description='Left Product:', style={'description_width': 'initial'})
dropdown_right = ipyw.Dropdown(options=list(zip(products, range(1, len(products)+1))), value=products.index(right_product)+1, description='Right Product:', style={'description_width': 'initial'})

toggle_products = [ipyw.Checkbox(value=True, description=label, layout=ipyw.Layout(width='40%')) for label in products] # Create checkboxes for products to use
toggle_col_clip = ipyw.Checkbox(value=False, description="Clip to modis", layout=ipyw.Layout(width='40%'))
date_left = ipyw.DatePicker(description='Start Date:', disabled=False, value = start_date) # Create datepicker for selected start date
date_right = ipyw.DatePicker(description='End Date:', disabled=False, value = end_date) # Create datepicker for selected end date


# ===== Initialize widgets, layers and controls =====
geo_json_map, vmin, vmax = create_choropleth(df=data, key_column="name", value_column=metric+"_"+f"{left_product} x {right_product}", metric=metric) # Initial choropleth map layer
legend = create_legend(vmin, vmax, metric=metric) # Create Legend for choropleth
plot = create_plot_as_widget(station, products, start_date=start_date, end_date=end_date) # Timeseries plot
tab_static = create_static_table_as_widget() # Static table
tab_dynamic = create_dynamic_table_as_widget(station=station, df=time_filtered_df) # Dynamic table


# ===== Define dashboard behavior =====
m.add_layer(geo_json_map)
m.add_control(legend)
geo_json_map.on_click(on_click)

metric_buttons.observe(update_choropleth_product, "value")
dropdown_left.observe(update_choropleth_product, "value")
dropdown_right.observe(update_choropleth_product, "value")

for p in range(len(products)):
    toggle_products[p].observe(update_timeseries_products, "value")
toggle_col_clip.observe(switch_dataframe, "value")
date_left.observe(update_timeseries_dates, "value")
date_right.observe(update_timeseries_dates, "value")


# ===== Set the layout of the dashboard =====
box_layout = ipyw.Layout(display='flex', flex_flow='column', align_items='center', width='100%')

lbox = ipyw.VBox([ipyw.HBox([dropdown_left, dropdown_right]), metric_buttons, m, tab_static], layout=box_layout)
rbox = ipyw.VBox([ipyw.HBox([date_left, date_right, toggle_col_clip]), ipyw.HBox(children=toggle_products), plot, tab_dynamic], layout=box_layout)
hbox = ipyw.HBox([lbox, rbox])
hbox
